In [106]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import boto3
import time
from datetime import datetime, timedelta
import pytz
from plot_opening_lines import list_filenames, plot_moneylines


In [157]:
def list_filenames(bucket_name, directory_prefix):
    # Create a boto3 client for S3
    s3 = boto3.client('s3')

    # Initialize a paginator to handle multiple pages of S3 objects (if necessary)
    paginator = s3.get_paginator('list_objects_v2')

    # Create a list to hold the filenames
    filenames = []

    # Use the paginator to fetch all objects within the specified directory
    for page in paginator.paginate(Bucket=bucket_name, Prefix=directory_prefix):
        # Access the 'Contents' from each page which lists the objects
        for obj in page.get('Contents', []):  # Check if 'Contents' key is in page
            filenames.append(obj['Key'])

    return filenames
#Usage
bucket_name = 'moneygans-data'  # Replace with your S3 bucket name
directory_prefix = 'basketball_nba/starting_money/'
#2024-11-01/live_odds/'
# 'baseball_mlb/aggregated_data/2024-07-24_all/'  # Ensure this ends with a slash if you're specifying a folder

# Get the list of filenames
file_list = list_filenames(bucket_name, directory_prefix)
file_list[::2][:-2]
# df = pd.read_parquet(f's3://moneygans-data/{file_list[1]}', engine='pyarrow')

['basketball_nba/starting_money/2024-11-01_starting/data/starting_moneylines.parquet',
 'basketball_nba/starting_money/2024-11-02_starting/data/starting_moneylines.parquet',
 'basketball_nba/starting_money/2024-11-03_starting/data/starting_moneylines.parquet',
 'basketball_nba/starting_money/2024-11-04_starting/data/starting_moneylines.parquet',
 'basketball_nba/starting_money/2024-11-06_starting/data/starting_moneylines.parquet',
 'basketball_nba/starting_money/2024-11-07_starting/data/starting_moneylines.parquet',
 'basketball_nba/starting_money/2024-11-08_starting/data/starting_moneylines.parquet']

In [166]:
def combine_opening_money(df_all_path, df_new):
    df_all = pd.read_parquet(df_all_path, engine='pyarrow')
    df_new_combined = pd.concat([df_all, df_new], axis=0)
    return df_new_combined

df = pd.DataFrame()
for file in file_list[2::2][:-2]:
    df_new= pd.read_parquet(f's3://moneygans-data/{file}', engine='pyarrow')
    df_new['date'] = df_new['commence_time_pst'].apply(lambda x: x.split('T')[0]).unique()[0]
    df = pd.concat([df, df_new], axis=0)

    

In [167]:
OPENING_PATH = f's3://moneygans-data/basketball_nba/starting_money/starting_money_agg/starting_money_all.parquet'
df.to_parquet(OPENING_PATH, engine='pyarrow')


In [ ]:
def combine_opening_money(df_new):
    OPENING_PATH = f's3://moneygans-data/basketball_nba/starting_money/starting_money_all.parquet'
    df_all = pd.read_parquet(OPENING_PATH, engine='pyarrow')
    df_new['date'] = df_new['commence_time_pst'].apply(lambda x: x.split('T')[0]).unique()[0]
    df_new_combined = pd.concat([df_all, df_new], axis=0)
    df_new_combined.to_parquet(OPENING_PATH, engine='pyarrow')

In [ ]:
plot_moneylines(df, '2024-11-10', plot_agg=True)

/Users/jake/Desktop/Thesis/MoneyLineGANs/plot_opening_lines.py:61: FutureWarning: 

`shade` is now deprecated in favor of `fill`; setting `fill=True`.
This will become an error in seaborn v0.14.0; please update your code.

  sns.kdeplot(
/Users/jake/opt/anaconda3/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/jake/opt/anaconda3/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/jake/opt/anaconda3/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before oper

File '2024-11-10_starting_moneyline_plots.pdf' uploaded to 's3://moneygans-data/basketball_nba/starting_money/starting_money_agg/plots/2024-11-10_starting_moneyline_plots.pdf' successfully.


In [121]:
def get_current_date_time_est():
    # Define the Pacific Standard Time zone
    pst_timezone = pytz.timezone('America/Los_Angeles')
    
    # Get the current time in UTC
    utc_time = datetime.now(pytz.utc)
    
    # Convert the current UTC time to PST time
    pst_time = utc_time.astimezone(pst_timezone)
    
    return pst_time

prev_date = (get_current_date_time_est() - timedelta(days=1)).strftime('%Y-%m-%d')

In [124]:
directory_prefix = f'basketball_nba/starting_money/{prev_date}_starting/data/'
list_filenames(bucket_name, directory_prefix)[0]

'basketball_nba/starting_money/2024-11-09_starting/data/starting_moneylines.parquet'

In [169]:
pd.read_parquet(OPENING_PATH, engine='pyarrow')['date'].unique()

array(['2024-11-02', '2024-11-03', '2024-11-04', '2024-11-06',
       '2024-11-07', '2024-11-08', '2024-11-09'], dtype=object)